In [1]:
import gpytorch
import torch

import cnn_gp

In [2]:
import torchvision

In [3]:
import sys
sys.path.append('..')
from kernels.cnn_kernel import CNNGP_Kernel

In [4]:
### load dataset
mnist = torchvision.datasets.MNIST('/home/wesley/Documents/datasets/', download=True, 
                                   transform=torchvision.transforms.ToTensor())

In [5]:
### construct training and test datasets (batched for now)

iter_data_loader = iter(torch.utils.data.DataLoader(mnist, batch_size = 128))

inputs, targets = next(iter_data_loader)
targets = targets.float()

if torch.cuda.is_available():
    inputs, targets = inputs.cuda(), targets.cuda()
    
test_inputs, test_targets = next(iter_data_loader)
test_targets = test_targets.float()

if torch.cuda.is_available():
    test_inputs, test_targets = test_inputs.cuda(), test_targets.cuda()

In [6]:
### define model from resnet on mnist
in_channels = 1
out_channels = 10

var_bias = 0.86
var_weight = 0.79

layers = []
for _ in range(7):  # n_layers
    layers += [
        cnn_gp.Conv2d(kernel_size=7, padding="same", var_weight=var_weight,
               var_bias=var_bias),
        cnn_gp.ReLU(),
    ]
initial_model = cnn_gp.Sequential(
    *layers,
    cnn_gp.Conv2d(kernel_size=28, padding=0, var_weight=var_weight,
           var_bias=var_bias),
)

In [7]:
class DPClassificationModel(gpytorch.models.ExactGP):
    def __init__(self, cnn_model, input_shape, train_x, train_y, alpha_epsilon = 0.01):
        num_classes = train_y.max() + 1
        
        # set alpha = \alpha_\epsilon
        alpha = alpha_epsilon * torch.ones(train_x.shape[-2], num_classes, 
                                           device = train_x.device, dtype = train_x.dtype)
        
        # alpha[class_labels] = 1 + \alpha_\epsilon
        alpha[torch.arange(len(train_x)), train_y] = alpha[torch.arange(len(train_x)), train_y] + 1.
        
        # sigma^2 = log(1 / alpha + 1)
        sigma2_i = torch.log(1 / alpha + 1.)
        
        # y = log(alpha) - 0.5 * sigma^2
        transformed_targets = alpha.log() - 0.5 * sigma2_i
        
        likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(sigma2_i.t(), learn_additional_noise= True)
        
        super(DPClassificationModel, self).__init__(train_x, transformed_targets.t(), likelihood)
        self.transformed_targets = transformed_targets
        
        self.mean_module = gpytorch.means.ConstantMean(batch_size = num_classes)
        self.covar_module = CNNGP_Kernel(cnn_model, input_shape, batch_shape=torch.Size((num_classes,)))
        
        self.likelihood = likelihood
    
    def forward(self, x):
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)

In [8]:
### define model
### TODO: check that our 2d reshaping is reasonable

#ikelihood = gpytorch.likelihoods.GaussianLikelihood()
model = DPClassificationModel(initial_model, inputs.shape[1:], train_x = inputs.view(inputs.shape[0], 28*28), 
                              train_y=targets.long())
likelihood = model.likelihood

if torch.cuda.is_available():
    likelihood = likelihood.cuda()
    model = model.cuda()

In [9]:
%pdb

Automatic pdb calling has been turned ON


In [10]:
model(inputs.view(inputs.shape[0], 28*28)).rsample()

torch.Size([10]) False
shape of kernel is:  torch.Size([128, 128])
<gpytorch.lazy.non_lazy_tensor.NonLazyTensor object at 0x7f89709eef10> torch.Size([128, 128])
inside:  torch.Size([1, 128, 128])
<gpytorch.lazy.non_lazy_tensor.NonLazyTensor object at 0x7f88f6c0a850> torch.Size([1, 128, 128])
<gpytorch.lazy.non_lazy_tensor.NonLazyTensor object at 0x7f88f6c0a850> torch.Size([1, 128, 128])
<gpytorch.lazy.non_lazy_tensor.NonLazyTensor object at 0x7f88f6c0a850> torch.Size([1, 128, 128])


/home/wesley/Documents/Code/gpytorch/gpytorch/utils/cholesky.py:42: RuntimeWarning: A not p.d., added jitter of 9.999999999999999e-06 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new} to the diagonal", RuntimeWarning)


RuntimeError: number of dims don't match in permute

> /home/wesley/Documents/Code/gpytorch/gpytorch/lazy/lazy_tensor.py(1593)zero_mean_mvn_samples()
   1591             *self.batch_shape, covar_root.size(-1), num_samples, dtype=self.dtype, device=self.device
   1592         )
-> 1593         samples = covar_root.matmul(base_samples).permute(-1, *range(self.dim() - 1)).contiguous()
   1594 
   1595         return samples

ipdb> q


In [ ]:
from botorch.optim.fit import fit_gpytorch_torch

In [ ]:
### define and fit any free parameters in the model

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
fit_gpytorch_torch(mll, options={"maxiter": 1000, "disp": True, "lr": 0.1})

In [ ]:
### set to test mode

model.eval()
likelihood.eval()

In [ ]:
test_preds = model(test_inputs.view(test_inputs.shape[0], 28*28))

In [ ]:
test_preds.mean

In [ ]:
### yikes :(

In [12]:
a = torch.randn(513, 513)
symm_a = a.t() @ a
covar = gpytorch.lazy.BatchRepeatLazyTensor(gpytorch.lazify(symm_a), torch.Size((10,)))
mean = torch.randn(10, 30)

dist = gpytorch.distributions.MultivariateNormal(mean, covar)
dist.rsample()

<gpytorch.lazy.non_lazy_tensor.NonLazyTensor object at 0x7f88f00fc590> torch.Size([30, 30])
inside:  torch.Size([1, 30, 30])
<gpytorch.lazy.non_lazy_tensor.NonLazyTensor object at 0x7f88f04ea5d0> torch.Size([1, 30, 30])
<gpytorch.lazy.non_lazy_tensor.NonLazyTensor object at 0x7f88f010df90> torch.Size([1, 30, 30])
<gpytorch.lazy.non_lazy_tensor.NonLazyTensor object at 0x7f88f010df90> torch.Size([1, 30, 30])
<gpytorch.lazy.non_lazy_tensor.NonLazyTensor object at 0x7f88f010df90> torch.Size([1, 30, 30])
<gpytorch.lazy.batch_repeat_lazy_tensor.BatchRepeatLazyTensor object at 0x7f88f00fc750> torch.Size([10, 30, 30])


tensor([[ -1.8400,   0.3161,  -0.7144,  -0.3811,  -2.6392,   9.1203, -15.6471,
           6.8704,   3.8084,  -1.1872, -11.0476,  -2.6245,  -8.4925,  16.3853,
          -4.7919,  -5.1416,   0.8967,  -0.2628,  14.0446,   5.1566,  -6.2106,
         -11.7443,  -7.3664,   7.7204,  -0.7164, -11.3812,   3.0444,   6.4354,
           0.0371, -11.3726],
        [ -2.4838,   7.8087,  -5.7541,   4.9832,  -4.8660,   7.4775, -13.1246,
           5.0781,   1.5952,  -2.6066,   5.6760,   5.3088,   6.1152,  -2.1733,
          -2.0447,   2.7258,  12.3781,  -0.5361,  -4.8066,   0.3408,  -0.1203,
           2.9489,  -8.4204,  13.0459,   7.4378,   6.4842,  -2.9404,  -2.6798,
           3.5718,   4.4365],
        [  5.9035,   3.5853,  -6.6984,   8.7704,   2.8835,   5.8287,  -7.8802,
           4.2365,  -6.2981,  -1.8538,   9.5167,  -4.6293,   7.8201,   3.0214,
          -8.4342,  -0.6910,   1.7082,   7.7967, -10.2497,   3.3097,   2.9738,
          -7.1227,  -0.7410,   4.4709,   3.2216,  -0.2859,  -3.2544,  -

In [ ]:
x.dim()